# Exploratory Data Analysis

Exploratory Data Analysis (EDA) is conducted to gain insights and understand the underlying patterns, relationships, and characteristics of a dataset. It helps in forming initial hypotheses, identifying data quality issues, and guiding further analysis. Here are some common questions that EDA aims to answer:

- **What are the main features and variables in the dataset?** EDA helps in understanding the structure of the data and identifying the different variables and their types.

- **What is the distribution of the variables?** EDA allows for the examination of the data distribution, such as checking for normality, skewness, or multimodality, which can provide insights into the nature of the variables.

- **Are there any missing values or data quality issues?** EDA helps in detecting missing values, outliers, inconsistencies, or other data quality problems that may require cleaning or imputation.

- **What is the relationship between variables?** EDA explores the associations and correlations between variables, which can uncover patterns, dependencies, or potential predictive relationships.

- **Are there any trends or patterns over time?** EDA investigates time-dependent data to identify trends, seasonality, cyclic patterns, or other temporal characteristics.

- **Are there any differences or patterns across different groups or categories?** EDA allows for the comparison and analysis of data based on different groups or categories, such as demographic variables or experimental conditions.

- **Are there any outliers or extreme observations?** EDA helps in identifying outliers or extreme values that may require further investigation to determine if they are valid data points or errors.

- **What are the summary statistics or descriptive measures of the variables?** EDA calculates summary statistics, such as mean, median, standard deviation, and percentiles, to describe the central tendency, variability, and shape of the data.

- **Are there any patterns or clusters in the data?** EDA explores the data for potential clusters or groups based on similarity or proximity to uncover hidden patterns or subpopulations.

- **What are the limitations or potential biases in the data?** EDA provides an opportunity to assess the limitations, biases, or sampling issues in the data, which helps in interpreting the results accurately.

These questions are not exhaustive, and the specific questions to be answered through EDA depend on the dataset, domain, and research or analysis objectives. EDA is a flexible and iterative process that allows for exploratory investigation and hypothesis generation before further analysis or modeling.


This notebook provides a starting template for exploratory data analysis.
Implementation details for EDA will vary based on the usecase but this notebook can be used as a starting point for any use case.
To use the notebook for a new use case, fill in the relevant blanks / parameters in the notebook or add relevant additional steps.

In [ ]:
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.impute import SimpleImputer

## Import Data

Data could be imported from Google Cloud Storage (GCS), local csv file or Bigquery. Managing data via Bigquery is the recommended appraoch for tabular data as it is easier to maintain and update the data as compared to csv files stores locally or in GCs. For other data types like images or text, GCS will be the input source.

For importing data from Bigquery, we can import the necessary modules and functions directly into the notebook from src. These functions can be quickly and easily accessed and run with defined input parameters. Their output can be examined and used as inputs for other functions. With this the ML training, evaluation or inference flow can be easily run using the already refactored code.

Below is the sample for importing data from Bigquery

In [ ]:
# import relevant libraries
from xgb_churn_prediction.data import data_ingestion

In [ ]:
# Define constants

# our GCP project ID - currently 'ncau-data-nprod-aitrain'
PROJECT_ID="nca-datapl-nprod-dev-genai"
# the region of our gcp project
LOCATION="us-central1"

In [ ]:
# Set up the BigQuery query
data_query = f"""
SELECT *
FROM `project_id.your_dataset.your_table`
"""

"""Optional Dictionary to set the dataype mapping when reading the data
Dictionary format: <column_name>:<datatype>"""

dtypes = {'column_name1': int, 'column_name2': float}

data_df = data_ingestion.execute_bq_query(PROJECT_ID, data_query, dtypes)

### Dataset overview

In [ ]:
# Print number of rows, columns
print(data_df.shape)

# Print datatype for columns and index information
print(data_df.info())

# Look at first 10 rows of dataframe
data_df.head(10)

**Observation** : Log any relevant comments / observations if applicable


### Descriptive statistics for all numeric columns in the dataset

In [ ]:
data_df.describe()

**Observation** : Log any relevant comments / observations if applicable



## Data Cleaning 

### Checking for null values

Caculating percentage of rows with null values for each column

In [ ]:
print(data_df.isna().mean() * 100)

### Handle null values

There are several ways to handle missing data in Python. Here are some common techniques:

1. Dropping missing values:

    - Use dropna() function from pandas to remove rows or columns with missing values. For example, data.dropna() will drop all rows containing any missing values.
    - You can specify the axis and threshold for dropping. For example, data.dropna(axis=1, thresh=1000) will drop columns that have less than 1000 non-null values.
   
2. Filling missing values:

    - Use fillna() function from pandas to fill missing values with a specific value or strategy. For example, data.fillna(0) will fill all missing values with 0.
    - You can also use statistical measures like mean, median, or mode to fill missing values. For example, data['column1'].fillna(data['column1'].mean()) will fill missing values in a specific column with the mean value of that column.

3. Interpolation

    - Use interpolation methods to estimate missing values based on the existing data. Pandas provides various interpolation methods such as linear, quadratic, and cubic. For example, data['column1'].interpolate(method='linear') will fill missing values using linear interpolation in a specific column.

4. Imputation:

    - Use advanced imputation techniques to estimate missing values based on the relationships between variables. The scikit-learn library provides the SimpleImputer class for imputation.
    

It's important to choose the appropriate method based on the characteristics of your dataset and the nature of the missing data. Consider the amount of missing data, the impact of missing values on your analysis, and the assumptions you can make about the data when deciding how to handle missing data.

Below are some methods that can be used for handling missing values based on the usecase

In [ ]:
def drop_missing_values(dataframe, axis=0, thresh=None):
    # Drop rows or columns with missing values
    return dataframe.dropna(axis=axis, thresh=thresh)

def fill_missing_values(dataframe, value=None, method=None):
    # Fill missing values with a specific value or strategy
    if value is not None:
        return dataframe.fillna(value)
    elif method is not None:
        if method == 'mean':
            return dataframe.fillna(dataframe.mean())
        elif method == 'median':
            return dataframe.fillna(dataframe.median())
        elif method == 'mode':
            return dataframe.fillna(dataframe.mode().iloc[0])
        else:
            raise ValueError("Invalid method. Choose 'mean', 'median', or 'mode'.")
    else:
        raise ValueError("Specify either a value or a method.")

def interpolate_missing_values(dataframe, method='linear'):
    # Interpolate missing values using interpolation methods
    return dataframe.interpolate(method=method)

def impute_missing_values(dataframe, strategy='mean'):
    # Impute missing values using advanced imputation techniques
    imputer = SimpleImputer(strategy=strategy)
    return pd.DataFrame(imputer.fit_transform(dataframe), columns=dataframe.columns)

In [ ]:
data_df = drop_missing_values(data_df)

### Check for missing data for specific dates (Only applicable for time series data)

If your time series data is expected to have a regular frequency (e.g., daily, monthly), checking for missing dates is important. Missing dates can introduce gaps in the time series, affecting the accuracy of any time-based calculations or analyses. In such cases, it's crucial to identify and handle missing dates appropriately.

If your time series data comes from a data collection process where missing dates are expected or occur naturally, it might be acceptable to have missing dates. For example, if you have daily sales data, missing dates on weekends or public holidays could be considered normal.

In [ ]:
def check_missing_dates(dataframe, date_column, group_column=None):
    """
    Check for missing dates in a time series DataFrame.

    Parameters:
        dataframe (pandas.DataFrame): The DataFrame containing the time series data.
        date_column (str): The name of the column representing the dates.
        group_column (str, optional): The name of the column to group the data by (default: None).

    Returns:
        None

    Prints the missing dates in the time series data. If `group_column` is None, missing dates
    are checked for the entire DataFrame. If `group_column` is specified, missing dates are
    checked for each group separately.
    
    Note:
        - The function converts the date column to datetime type if necessary.
        - The function uses pandas date_range to create a complete date range based on the minimum
          and maximum dates in the DataFrame or group.
        - Missing dates are identified by comparing the complete date range with the DataFrame's
          or group's date column using the isin method.

    Example:
        data = {
            'date': ['2023-01-01', '2023-01-02', '2023-01-04', '2023-01-01', '2023-01-03', '2023-01-04'],
            'value': [10, 20, 30, 40, 50, 60],
            'group': ['A', 'A', 'A', 'B', 'B', 'B']
        }
        df = pd.DataFrame(data)
        check_missing_dates(df, 'date')
        print()
        check_missing_dates(df, 'date', 'group')
    """
    # Convert the date column to datetime type if necessary
    if not pd.core.dtypes.common.is_datetime_or_timedelta_dtype(dataframe[date_column]):
        dataframe[date_column] = pd.to_datetime(dataframe[date_column])
        
    if group_column is None:
        # If no group column is specified, check missing dates for the entire DataFrame
        complete_date_range = pd.date_range(start=dataframe[date_column].min(), end=dataframe[date_column].max(), freq='D')

        missing_dates = complete_date_range[~complete_date_range.isin(dataframe[date_column])]

        if missing_dates.empty:
            print("No missing dates found.")
        else:
            print("Missing dates:")
            print(missing_dates)

    else:
        # If a group column is specified, check missing dates for each group
        groups = dataframe.groupby(group_column)

        for group_name, group_data in groups:
            print(group_data)
            complete_date_range = pd.date_range(start=group_data[date_column].min(), end=group_data[date_column].max(), freq='D')

            missing_dates = complete_date_range[~complete_date_range.isin(group_data[date_column])]

            if missing_dates.empty:
                print(f"No missing dates found for group: {group_name}")
            else:
                print(f"Missing dates for group: {group_name}")
                print(missing_dates)

In [ ]:
check_missing_dates(data_df, 'date_column_name')

**Observation** : Log any relevant comments / observations if applicable

### Check for outliers

There are several approaches to check for outliers in a dataset. Here are some commonly used methods:

1. **Z-Score Method**: Calculate the z-score for each data point by subtracting the mean and dividing by the standard deviation. Data points with z-scores above a certain threshold (e.g., 2 or 3) are considered outliers.

2. **Modified Z-Score Method**: Similar to the z-score method, but instead of using the mean and standard deviation, it uses the median and median absolute deviation (MAD) to calculate the modified z-score. Data points with modified z-scores above a threshold (e.g., 2.5 or 3) are considered outliers.

3. **Percentiles/Quartiles**: Compute the lower and upper quartiles and the interquartile range (IQR). Data points below the lower quartile minus a certain multiplier times the IQR or above the upper quartile plus a certain multiplier times the IQR are considered outliers. Common multipliers are 1.5 or 3.

4. **Box Plots**: Create box plots to visualize the distribution of the data. Outliers are typically identified as points that fall outside the whiskers of the box plot.

5. **Histograms**: Generate histograms or density plots to visualize the frequency distribution of the data. Unusually high or low values that appear far away from the bulk of the data can be potential outliers.

6. **Domain Knowledge and Business Rules**: Apply domain-specific knowledge or business rules to identify outliers. For example, in a sales dataset, unusually high or low sales figures may be considered outliers based on predetermined rules.

7. **Machine Learning Models**: Train a machine learning model (e.g., linear regression, random forest) and examine the residuals or errors. Unusually large residuals or errors can indicate the presence of outliers.

8. **Cluster Analysis**: Perform cluster analysis to group similar data points together. Outliers can be identified as points that do not belong to any cluster or form their own separate cluster.

It is important to note that no single method is foolproof, and the choice of approach depends on the nature of the data, the specific problem domain, and expert knowledge. It is often recommended to use multiple methods and consider the context of the data before determining which data points are truly outliers.

Below is an implementation plotting the box plot and histogram and calculating the outliers based on modified Z Score as an example

In [ ]:
def check_outliers(dataframe, column, group_column=None):
    """
    Check for outliers in a DataFrame column, optionally aggregated by a group column.

    Parameters:
        dataframe (pandas.DataFrame): The DataFrame containing the data.
        column (str): The name of the column to check for outliers.
        group_column (str, optional): The name of the column to group the data by (default: None).

    Returns:
        None

    Calculates summary statistics, creates box plots and histograms, and identifies outliers
    using z-score and modified z-score methods. Outliers are printed for each group if
    `group_column` is specified.

    Example:
        data = {
            'group': ['A', 'A', 'B', 'B', 'B', 'C', 'C'],
            'value': [10, 20, 5, 30, 40, 100, 110]
        }
        df = pd.DataFrame(data)
        check_outliers(df, 'value')
        print()
        check_outliers(df, 'value', 'group')
    """
    if group_column is None:
        # Calculate summary statistics
        summary_stats = dataframe[column].describe()
        print("Summary Statistics:")
        print(summary_stats)
        print()

        # Create box plot
        plt.figure(figsize=(6, 4))
        sns.boxplot(data=dataframe, y=column)
        plt.title("Box Plot")
        plt.show()

        # Create histogram
        plt.figure(figsize=(6, 4))
        sns.histplot(data=dataframe, x=column, kde=True)
        plt.title("Histogram")
        plt.show()

        # Calculate z-scores
        z_scores = (dataframe[column] - np.mean(dataframe[column])) / np.std(dataframe[column])

        # Define threshold for z-score method
        z_score_threshold = 3

        # Identify outliers using z-score method
        outliers_zscore = dataframe[np.abs(z_scores) > z_score_threshold]
        print("Outliers (Z-score method):")
        print(outliers_zscore)
    else:
        # Group by the specified column
        grouped_data = dataframe.groupby(group_column)

        for group_name, group_data in grouped_data:
            print(f"Group: {group_name}")
            print("-------------")

            # Calculate summary statistics for each group
            summary_stats = group_data[column].describe()
            print("Summary Statistics:")
            print(summary_stats)
            print()

            # Create box plot for each group
            plt.figure(figsize=(6, 4))
            sns.boxplot(data=group_data, x=group_column, y=column)
            plt.title(f"Box Plot - Group {group_name}")
            plt.show()

            # Create histogram for each group
            plt.figure(figsize=(6, 4))
            sns.histplot(data=group_data, x=column, kde=True)
            plt.title(f"Histogram - Group {group_name}")
            plt.show()

            # Calculate z-scores for each group
            z_scores = (group_data[column] - np.mean(group_data[column])) / np.std(group_data[column])

            # Define threshold for z-score method
            z_score_threshold = 3

            # Identify outliers using z-score method for each group
            outliers_zscore = group_data[np.abs(z_scores) > z_score_threshold]
            print("Outliers (Z-score method):")
            print(outliers_zscore)

            print("\n")

In [ ]:
check_outliers(data_df, 'value_column_name')

**Observation** : Log any relevant comments / observations if applicable

### Handling outliers


Handling outliers depends on the specific context and goals of your analysis. Here are some common approaches for dealing with outliers:

- Leave the Outliers: In some cases, outliers may be valid and meaningful data points. If you have confidence in the accuracy and relevance of the outliers, you can choose to leave them as they are and proceed with your analysis. However, it's important to interpret the results with caution, as outliers can have a significant impact on statistical measures and models.

- Remove the Outliers: If the outliers are due to data entry errors, measurement errors, or other anomalies that are unlikely to represent true values, you can remove them from your dataset. However, it's crucial to document and justify the reason for their removal to ensure transparency and reproducibility of your analysis.

- Transform the Data: Instead of removing outliers, you can apply data transformations to make the data distribution more suitable for analysis. For example, you can use logarithmic or square root transformations to reduce the impact of extreme values while preserving the overall patterns in the data.

- Imputation: If you decide to keep the outliers in your dataset, you can consider imputing their values instead of removing them. Imputation involves estimating missing or extreme values based on other data points or statistical techniques. Various imputation methods, such as mean imputation, median imputation, or regression imputation, can be used depending on the characteristics of your data.

- Separate Analysis: In some cases, outliers may represent a distinct subgroup within your data that requires separate analysis. You can consider analyzing the outliers separately or applying different techniques tailored to their specific characteristics.

It's important to remember that the appropriate approach for handling outliers depends on the specific dataset, the analysis objectives, and the domain knowledge. It's recommended to consider the impact of outlier handling on your analysis results and validate the robustness of your findings with and without outlier treatment.

Below are some methods that showcase how to apply some of these techniques.

In [ ]:
def handle_outliers_remove(dataframe, column, method='iqr', multiplier=1.5):
    # Remove outliers using specified method and multiplier
    if method == 'iqr':
        q1 = dataframe[column].quantile(0.25)
        q3 = dataframe[column].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - multiplier * iqr
        upper_bound = q3 + multiplier * iqr
        return dataframe[(dataframe[column] >= lower_bound) & (dataframe[column] <= upper_bound)]
    elif method == 'std':
        z_scores = (dataframe[column] - np.mean(dataframe[column])) / np.std(dataframe[column])
        return dataframe[np.abs(z_scores) <= multiplier]
    else:
        raise ValueError("Invalid method. Choose 'iqr' or 'std'.")

def handle_outliers_transform(dataframe, column, method='log'):
    # Apply data transformation to handle outliers
    if method == 'log':
        dataframe[column] = np.log1p(dataframe[column])
    elif method == 'sqrt':
        dataframe[column] = np.sqrt(dataframe[column])
    else:
        raise ValueError("Invalid method. Choose 'log' or 'sqrt'.")
    return dataframe

def handle_outliers_impute(dataframe, column, method='mean'):
    # Impute the outliers with specified method
    if method == 'mean':
        dataframe[column] = np.where(
            np.abs(dataframe[column] - np.mean(dataframe[column])) > 2 * np.std(dataframe[column]),
            np.mean(dataframe[column]),
            dataframe[column]
        )
    elif method == 'median':
        dataframe[column] = np.where(
            np.abs(dataframe[column] - np.median(dataframe[column])) > 2 * np.median(dataframe[column]),
            np.median(dataframe[column]),
            dataframe[column]
        )
    else:
        raise ValueError("Invalid method. Choose 'mean' or 'median'.")
    return dataframe

## Data Analysis and Visualisation

As part of Exploratory Data Analysis (EDA), various data visualizations can provide valuable insights into the dataset. Here are some common types of visualizations that can be helpful during EDA:

- **Histograms**: Histograms display the distribution of a numerical variable by dividing it into bins and showing the frequency or count of values within each bin. They help to understand the shape, central tendency, and spread of the data.

- **Box plots**: Box plots provide a summary of the distribution of a numerical variable by displaying the median, quartiles, and any outliers. They help identify the presence of outliers, skewness, and overall variability in the data.

- **Scatter plots**: Scatter plots show the relationship between two numerical variables. They help visualize patterns, correlations, and potential outliers. Scatter plots can also be enhanced with color or size encoding to represent additional dimensions.

- **Line plots**: Line plots are useful for visualizing trends and changes over time or other ordered categories. They can show how a variable evolves and reveal patterns, seasonality, or anomalies.

- **Bar charts**: Bar charts are suitable for visualizing categorical variables by displaying the count or proportion of each category. They help compare different categories and identify the most frequent or significant ones.

- **Heatmaps**: Heatmaps provide a visual representation of the relationship between two categorical variables. They use color to represent the frequency or proportion of each combination of categories, allowing the identification of associations or dependencies.

- **Correlation matrices**: Correlation matrices, often visualized as heatmaps, display the pairwise correlations between numerical variables. They help identify relationships, dependencies, and multicollinearity among variables.


These visualizations serve as a starting point, and the choice of specific plots depends on the **nature of the data** and **the questions being explored**. It's important to select visualizations that effectively represent the variables of interest and facilitate the identification of patterns, trends, outliers, and relationships within the dataset.

In [ ]:
# Histogram
plt.hist(data_df['column1'], bins=10)
plt.xlabel('X-axis label')
plt.ylabel('Y-axis label')
plt.title('Histogram of Column')
plt.show()

In [ ]:
# Box plot
sns.boxplot(x=data_df['column1'])
plt.xlabel('X-axis label')
plt.title('Box Plot of Column')
plt.show()

In [ ]:
# Scatter plot
plt.scatter(data_df['column1'], data_df['column2'])
plt.xlabel('X-axis label')
plt.ylabel('Y-axis label')
plt.title('Scatter Plot')
plt.show()

In [ ]:
# Correlation heatmap for numeric columns
numeric_columns = data_df.select_dtypes(include=[np.number])
corr_matrix = numeric_columns.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Bar chart
sns.countplot(data_df['column1'])
plt.xlabel('X-axis label')
plt.ylabel('Y-axis label')
plt.title('Bar Chart')
plt.show()

## EDA Summary and Conclusion


Concluding an Exploratory Data Analysis (EDA) notebook is an important step to summarize the insights gained from the analysis. Here are some key points to consider when concluding an EDA notebook:

- **Dataset Summary**: Provide an overview of the dataset, including its size, features, and relevant background information.

- **Data Quality and Cleaning**: Highlight any data quality issues encountered and the steps taken to clean and preprocess the data.

- **Key Findings**: Summarize the main insights and patterns discovered during the analysis.

- **Feature Importance**: Identify the most influential features that have a significant impact on the target variable.

- **Outliers and Anomalies**: Discuss any outliers or anomalies identified and their potential impact on the analysis.

- **Data Distributions**: Summarize the distribution characteristics of the data for each feature.

- **Data Limitations**: Acknowledge any limitations or caveats of the dataset that may affect the analysis or generalizability of the findings.

- **Next Steps**: Suggest potential next steps or areas of further investigation based on the EDA.

- **Conclusion**: Provide a concise summary of the key insights gained from the EDA.

- **Final Thoughts**: Share any additional reflections or recommendations based on the analysis.

Remember to keep the language clear and to the point, and provide supporting evidence or visualizations where necessary.